In [41]:
import numpy as np
import pandas as pd

import os
import warnings

In [129]:
#Utility function to delete last 3 characters from a string
def deleteChars(str):
    return str[:-3]

#Inner join wrapper function on two datasets
def dfIntersection(dataframe1, dataframe2):
    return pd.merge(dataframe1, dataframe2, on=["Code"], how="inner")

# Preprocessing + Combining Multiple Variables
def fetchOnetDB(variables):
    onetDB = pd.DataFrame()
    for location,heading in variables.items():
        data = pd.read_csv("data/"+location)
        if "Importance" in data.columns: 
            data.rename(columns={'Importance': heading}, inplace=True)
            del data['Level']
        if "Context" in data.columns: data.rename(columns={'Context': heading}, inplace=True)
        if onetDB.empty: onetDB = data.copy()
        else: onetDB = pd.merge(onetDB,data, on=['Code','Occupation'])
    cols = onetDB.columns.to_list()
    onetDB = onetDB[cols[1:3] + [cols[0]] + cols[3:]]
    onetDB["Code"] = onetDB["Code"].apply(deleteChars)
    return onetDB.copy()

#Function to fetch required columns from SOC data
def fetchSOCDB():
    socDb = pd.DataFrame()
#     df = pd.read_csv("data/soc_data_2019.csv", usecols = ["own_code", "occ_code", "occ_title", "tot_emp", "h_mean", "a_mean", "h_median", "a_median"])
#     df = pd.read_csv("data/all_data_M_2019.csv", usecols = ["own_code", "occ_code", "occ_title", "tot_emp", "loc_quotient", "pct_total", "h_mean", "a_mean", "h_median", "a_median"])
    df = pd.read_csv("data/all_data_M_2018.csv", usecols = ["own_code", "occ_code", "occ_title", "tot_emp", "loc_quotient", "pct_total", "h_mean", "a_mean", "h_median", "a_median"])
#     df = pd.read_csv("data/all_data_M_2017.csv", usecols = ["own_code", "occ_code", "occ_title", "tot_emp", "loc_quotient", "pct_total", "h_mean", "h_median"])
    df.rename(columns={'occ_code': "Code"}, inplace=True)
    df = df.drop_duplicates(subset=["occ_title"])
    return df.copy()

#Fetch Ofxord research data
def fetchOxfordDB():
    ofxfordDB = pd.DataFrame()
    df = pd.read_csv("data/oxford_data.csv")
    df.rename(columns={'code': "Code"}, inplace=True)
    return df

In [174]:
onetVars = {
    'Social_Perceptiveness.csv':'SP',
    'Persuasion.csv':'P',
    'Finger_Dexterity.csv':'FD',
    'Originality.csv':'O',
    'Assisting_and_Caring_for_Others.csv':'AC',
    'Manual_Dexterity.csv':'MD',
    'Cramped_Work_Space_Awkward_Positions.csv':'CW',
    'Negotiation.csv':'N',
    'Fine_Arts.csv':'FA'
}

onetDB = fetchOnetDB(onetVars)
print("ONET data Dimensions - "+str(onetDB.shape))

socDB = fetchSOCDB()
socDB['Code'] = socDB['Code'].astype(str)
print("SOC data Dimensions - "+str(socDB.shape))

oxfordDB = fetchOxfordDB()
print("Oxford data Dimensions - "+str(oxfordDB.shape))

warnings.filterwarnings('ignore')

ONET data Dimensions - (968, 11)
SOC data Dimensions - (1103, 10)
Oxford data Dimensions - (702, 5)


In [178]:
ONETSOCDB = pd.merge(socDB, onetDB, on=["Code"], how="inner")
print("ONET + SOC data intersection Dimensions - "+str(ONETSOCDB.shape))

onetOxfordDB = dfIntersection(oxfordDB, onetDB)
onetOxfordDB = onetOxfordDB.loc[onetOxfordDB["label"].notnull()]
print("Oxford + ONET data intersection Dimensions - "+str(onetOxfordDB.shape))

ONET + SOC data intersection Dimensions - (608, 20)
Oxford + ONET data intersection Dimensions - (84, 15)


In [179]:
ONETSOCDB = ONETSOCDB.drop(['loc_quotient','pct_total','occ_title'], axis=1)
ONETSOCDB[ONETSOCDB['Code'].str.contains('15-')]

,own_code,Code,tot_emp,h_mean,a_mean,h_median,a_median,Occupation,SP,P,FD,O,AC,MD,CW,N,FA
39,1235,15-1121,"587,970",45.01,"93,610",42.66,"88,740",Informatics Nurse Specialists,60,53,35,72,51,25,17,53,4
40,1235,15-1121,"587,970",45.01,"93,610",42.66,"88,740",Computer Systems Analysts,50,44,47,50,23,19,8,38,1
41,1235,15-1122,"108,060",49.26,"102,470",47.28,"98,350",Information Security Analysts,44,41,35,50,16,22,16,41,1
42,1235,15-1131,"230,470",43.07,"89,580",40.52,"84,280",Computer Programmers,53,41,31,50,29,25,1,28,14
43,1235,15-1132,"903,160",51.96,"108,080",49.82,"103,620","Software Developers, Applications",47,47,38,56,32,25,5,44,0
44,1235,15-1133,"405,330",54.81,"114,000",52.89,"110,000","Software Developers, Systems Software",53,44,38,53,21,19,6,38,6
45,1235,15-1134,"127,300",36.34,"75,580",33.38,"69,430",Web Developers,47,47,44,53,23,25,4,44,26
46,1235,15-1141,"110,090",44.25,"92,030",43.31,"90,070",Database Administrators,44,41,35,50,33,22,12,28,0
47,1235,15-1142,"366,250",41.86,"87,070",39.45,"82,050",Network and Computer Systems Administrators,50,44,44,53,44,38,26,38,2
48,1235,15-1143,"152,670",53.43,"111,130",52.41,"109,020",Telecommunications Engineering Specialists,56,50,47,60,31,38,38,47,1
